In [1]:
import os 
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd                      # 가장 기초적인 데이터전처리 패키지
import re                                # 정규표현식을 사용할 수 있는 패키지
from datetime import datetime  
import urllib.request
from PIL import Image
from pytesseract import *

In [2]:
data = pd.DataFrame({"date" : [],
                    "image" : [],
                    "name" : [],
                    "price" : [],
                    "link" : [],
                    "size":[]})

In [3]:
base_url = 'http://www.instylefit.com'

In [6]:
req = requests.get(base_url)
if req == 200 :
    soup = BeautifulSoup(req.content, 'html.parser')
else :
    req = requests.get(base_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup = BeautifulSoup(req.content, 'html.parser')

In [8]:
req

<Response [200]>

In [9]:
category_1 = soup.find('div',{'class' : 'xans-element- xans-layout xans-layout-category main_cate'})

In [10]:
category_2 = category_1.find_all('li',{'class' : 'xans-record-'})

In [12]:
category_list = []
for i in range(len(category_2)):
    category_lst = base_url +category_2[i].find('a')['href']
    category_list.append(category_lst)

In [13]:
category_list #추후에는 정규표현식을 사용해서 카테고리 사용할 카테고리 리스트만 뽑는 작업을 하자

['http://www.instylefit.com/category/당일출고/111/',
 'http://www.instylefit.com/category/신상-더-세일/318/',
 'http://www.instylefit.com/category/베스트/116/',
 'http://www.instylefit.com/category/가을신상/156/',
 'http://www.instylefit.com/category/outer/7/',
 'http://www.instylefit.com/category/top/12/',
 'http://www.instylefit.com/category/dress/52/',
 'http://www.instylefit.com/category/pants/114/',
 'http://www.instylefit.com/category/skirt/115/',
 'http://www.instylefit.com/category/bagshoesacc/312/',
 'http://www.instylefit.com/category/bikini/60/',
 'http://www.instylefit.com/category/fitness/51/',
 'http://www.instylefit.com/category/underwear/123/']

In [15]:
category_list2 = category_list[4:9]
start_url = category_list2[0]
page_url= category_list2[0] + '?page=' + '{}'

In [16]:
req2 = requests.get(start_url)
if req2 == 200 :
    soup2 = BeautifulSoup(req2.content, 'html.parser')
else :
    req2 = requests.get(start_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
    soup2 = BeautifulSoup(req2.content, 'html.parser')


In [17]:
page1_a = soup2.find('div',{'class':'xans-element- xans-product xans-product-normalpaging ec-base-paginate'})
page2_a = page1_a.find_all('li',{'class','xans-record-'})
    
page1_b = page1_a.find_all('a')[-1]['href'] #가장 마지막 페이지로 가는 url
page2_b = start_url + page1_b

In [19]:
if start_url+page2_a[-1].find('a')['href'] != page2_b:
    req3 = requests.get(page2_b)
    if req3 == 200 :
        soup3 = BeautifulSoup(req3.content, 'html.parser')
    else :
        req3 = requests.get(page2_b, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
        soup3 = BeautifulSoup(req3.content, 'html.parser')

    page3_b = soup3.find('div',{'class':'xans-element- xans-product xans-product-normalpaging ec-base-paginate'})
    page4_b = page3_b.find_all('li',{'class','xans-record-'})

    page5 = len(page2_a) + len(page4_b)
else:
    page5=len(page2_a)


In [20]:
page5

6

In [23]:
for i in range(1,page5+1):
    req4 = requests.get(page_url.format(str(i)))
    if req4 == 200 :
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    else :
        req4 = requests.get(page_url.format(str(i)), headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                            'Chrome/71.0.3578.98 Safari/537.36')})
        soup4 = BeautifulSoup(req4.content, 'html.parser')
    
    product_1 = soup4.find('ul',{'class','prdList grid4'})
    product_2 = product_1.find_all('li',{'class':'xans-record-'})

In [22]:
product_1

<ul class="prdList grid4">
<li class="xans-record-" id="anchorBoxId_26238">
<span class="chk"><input class="ProductCompareClass xECPCNO_26238 displaynone" type="checkbox"/></span>
<div class="thumbnail">
<a href="/product/미유카라코트/26238/category/7/display/1/" name="anchorBoxName_26238"><img alt="미유,카라코트" id="eListPrdImage26238_1" src="//www.instylefit.com/web/product/medium/201611/26238_shop1_208485.jpg"/><img alt="미유,카라코트" class="hover" id="" src="//www.instylefit.com/web/product/small/201611/26238_shop1_723266.jpg"/></a>
<ul class="button">
<li class="option"></li>
<li></li>
<li class="zoom"><img alt="상품 큰 이미지 보기" onclick="zoom('26238', '7', '1','', '');" src="/web/upload/icon_201808311514512700.png" style="cursor:pointer"/></li>
<li class="wish"><img alt="관심상품 등록 전" categoryno="7" class="icon_img ec-product-listwishicon" icon_status="off" individual-set="F" login_status="F" productno="26238" src="/web/upload/icon_201808311515038300.png"/></li>
</ul>
<div class="likeButton likePrd like

In [36]:
product_2[1]

<li class=" xans-record-">
<strong class="title displaynone"><span style="font-size:14px;color:#333333;font-weight:bold;">판매가</span> :</strong> <span style="font-size:14px;color:#333333;font-weight:bold;">89,900원</span><span id="span_product_tax_type_text" style=""> </span></li>

In [83]:
for product in product_2:
    try:
        link = product.find('div', {'class' : 'thumbnail'}).find('a')['href']
        link_url = base_url + link

        name = product.find('strong',{'class':'name'}).find_all('span')[2].text
        #해당 쇼피몰은 ':'문자열도 있기 때문에 2번째 위치에 제목이 있음
        name = re.sub('[^가-힣0-9a-zA-Z\\s\+\[\]\,\.\=\-\_\~\!\@\#\$\%\^\&\*\(\)\;\:\?]','',name)

        price_0 = product.find('ul', {'class' : 'xans-element- xans-product xans-product-listitem spec'}) # 가격정보가 있는 곳의 클래스
        price_1 = price_0.find('li', {'class' : 'xans-record-'}) # 원 가격, 할인가격 등 다양한 가격이 있는데, 할인가격으로 표시해주는 것
        price_2 = price_1.find_all('span')[1].text
        #해당 텍스트도 마찬가지 위치는 수동적으로 파악해야함 혹은 정규표현식 조건을 넣어주면 될듯

        req5 = requests.get(link_url)
        if req5 == 200 :
            soup5 = BeautifulSoup(req5.content, 'html.parser')
        else :
            req5 = requests.get(link_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
            soup5 = BeautifulSoup(req5.content, 'html.parser')

        image = soup5.find('div',{'class':'thumbnail'})
        image_url = image.find('img')['src']


        x= 'size_jpg'
        #soup5.find('div',{'class':'cont'}) #왜 못긁어오는거지... 
        #사이즈 이미지를 불러오는 과정에서 정보가 추출되지 않음 일부러 막아놨나.

        insert_data = pd.DataFrame({"date" : datetime.today().strftime("%Y-%m-%d"),
                                "image" : [image_url],
                                "name" : [name],
                                "price" : [price_2],
                                "link" : [link_url],
                                "size" : [x]})

        data = data.append(insert_data)
    except:
        pass

data = data.drop_duplicates(subset='image', keep='last') # 중복된 부분이 있다면 가장 최근것으로 대체
data.index = range(len(data))

In [84]:
data

,date,image,name,price,link,size
0,2019-03-08,//www.instylefit.com/web/product/big/201611/26...,"미유,카라코트","89,900원",http://www.instylefit.com/product/미유카라코트/26238...,size_jpg
1,2019-03-08,//www.instylefit.com/web/product/big/201603/24...,"러브잇,단추CD","37,500원",http://www.instylefit.com/product/러브잇단추cd/2484...,size_jpg
2,2019-03-08,//www.instylefit.com/web/product/big/201601/24...,"큐블,롱슬림JP","72,500원",http://www.instylefit.com/product/큐블롱슬림jp/2419...,size_jpg
3,2019-03-08,//www.instylefit.com/web/product/big/201511/23...,"파이션,포근퍼CD","45,000원",http://www.instylefit.com/product/파이션포근퍼cd/238...,size_jpg
4,2019-03-08,//www.instylefit.com/web/product/big/201510/23...,"조샵,도트집업","39,900원",http://www.instylefit.com/product/조샵도트집업/23628...,size_jpg
5,2019-03-08,//www.instylefit.com/web/product/big/201510/23...,"해볼드,트위드JK","59,500원",http://www.instylefit.com/product/해볼드트위드jk/235...,size_jpg
6,2019-03-08,//www.instylefit.com/web/product/big/201509/23...,"론드,플라워CD","35,500원",http://www.instylefit.com/product/론드플라워cd/2336...,size_jpg
7,2019-03-08,//www.instylefit.com/web/product/big/201412/20...,"패드,모직퍼JK","87,500원",http://www.instylefit.com/product/패드모직퍼jk/2018...,size_jpg
8,2019-03-08,//www.instylefit.com/web/product/big/201411/19...,"엣지트,훌CT","85,500원",http://www.instylefit.com/product/엣지트훌ct/19916...,size_jpg
9,2019-03-08,//www.instylefit.com/web/product/big/201411/19...,"파가온,카라배색CT","99,000원",http://www.instylefit.com/product/파가온카라배색ct/19...,size_jpg


In [59]:
for product in product_2:
    try:
        link = product.find('div', {'class' : 'thumbnail'}).find('a')['href']
        link_url = base_url + link

        req5 = requests.get(link_url)
        if req5 == 200 :
            soup5 = BeautifulSoup(req5.content, 'html.parser')
        else :
            req5 = requests.get(link_url, headers = {'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                                                'Chrome/71.0.3578.98 Safari/537.36')})
            soup5 = BeautifulSoup(req5.content, 'html.parser')

        image = soup5.find('div',{'class':'thumbnail'})
        image_url = image.find('img')['src']

        
        #soup5.find('div',{'class':'cont'}) #왜 못긁어오는거지... 
        #사이즈 이미지를 불러오는 과정에서 정보가 추출되지 않음 일부러 막아놨나.
        src=[]
        size = soup5.find_all('div',{'class':'-component-block'})
        size_img = size.find_all('img')

        for i in range(len(size_img)) :
            img_src = size_img[i]['src'] # 이미지의 주소에 대한 정보는 img 태그 내의 src 속성에 있다. 이를 가지고 오는 것.
            src.append(img_src)

        for result in src :
                if "size" in result :
                    x = result
                else:
                    pass
    except:
        pass